In [4]:
# Excel 2010 PT-BR workbook WITH formulas using commas as separators
# Uses SOMARPRODUTO and PT-BR function names with commas
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils import get_column_letter

rows = [
    [1, 22, 0],
    [2, 25, 0],
    [3, 28, 0],
    [4, 35, 1],
    [5, 47, 1],
    [6, 52, 1],
]

bold = Font(b=True)
center = Alignment(horizontal='center')
header_fill = PatternFill(start_color='FFEFEFEF', end_color='FFEFEFEF', fill_type='solid')
highlight_fill = PatternFill(start_color='FFDFF6DD', end_color='FFDFF6DD', fill_type='solid')
thin = Side(style='thin', color='FFB0B0B0')
border = Border(top=thin, left=thin, right=thin, bottom=thin)

wb = Workbook()

# Guia
ws = wb.active
ws.title = 'Guia'
ws['A1'] = 'CART (Árvore de Decisão) — Passo a Passo (Excel 2010 PT-BR, vírgulas)'
ws['A1'].font = Font(b=True, size=14)
steps = [
    '1) Dados: Idade e Classe.',
    '2) Thresholds: idades ordenadas e pontos médios (cortes).',
    '3) Contagens: SOMARPRODUTO sem coerção (sem --).',
    '4) Gini: proporções e Gini ponderado.',
    '5) Resumo: menor Gini determina T* e a regra.'
]
for i, s in enumerate(steps, start=3):
    ws[f'A{i}'] = s
ws.column_dimensions['A'].width = 110

# Dados
ws2 = wb.create_sheet('Dados')
ws2.append(['ID', 'Idade', 'Classe'])
for c in range(1, 4):
    cell = ws2.cell(row=1, column=c)
    cell.font = bold; cell.alignment = center; cell.fill = header_fill; cell.border = border
for r in rows:
    ws2.append(r)
for col in range(1, 4):
    ws2.column_dimensions[get_column_letter(col)].width = 12
for row in ws2.iter_rows(min_row=2, max_col=3):
    for cell in row:
        cell.border = border

# Thresholds
idades = sorted([r[1] for r in rows])
midpoints = [(idades[i] + idades[i+1]) / 2 for i in range(len(idades)-1)]
ws3 = wb.create_sheet('Thresholds')
ws3.append(['Idades ordenadas', 'Thresholds (pontos médios)'])
for c in range(1, 3):
    cell = ws3.cell(row=1, column=c)
    cell.font = bold; cell.alignment = center; cell.fill = header_fill; cell.border = border
for i, age in enumerate(idades, start=2):
    ws3.cell(row=i, column=1, value=age).border = border
for i, thr in enumerate(midpoints, start=2):
    ws3.cell(row=i, column=2, value=thr).border = border
ws3.column_dimensions['A'].width = 18
ws3.column_dimensions['B'].width = 24

# Contagens (SOMARPRODUTO sem --)
ws4 = wb.create_sheet('Contagens')
headers = ['Threshold T', 'Left_0 (<=T & Classe=0)', 'Left_1 (<=T & Classe=1)',
           'Right_0 (>T & Classe=0)', 'Right_1 (>T & Classe=1)',
           'n_left', 'n_right', 'n_total']
ws4.append(headers)
for idx in range(1, len(headers)+1):
    cell = ws4.cell(row=1, column=idx)
    cell.font = bold; cell.alignment = center; cell.fill = header_fill; cell.border = border

idade_rng = 'Dados!$B$2:$B$7'
classe_rng = 'Dados!$C$2:$C$7'

for i in range(2, 2+len(midpoints)):
    ws4.cell(row=i, column=1, value=f"=Thresholds!B{i}").border = border
    ws4.cell(row=i, column=2, value=f"=SOMARPRODUTO(("+idade_rng+f"<=A{i})*("+classe_rng+"=0))").border = border
    ws4.cell(row=i, column=3, value=f"=SOMARPRODUTO(("+idade_rng+f"<=A{i})*("+classe_rng+"=1))").border = border
    ws4.cell(row=i, column=4, value=f"=SOMARPRODUTO(("+idade_rng+f">A{i})*("+classe_rng+"=0))").border = border
    ws4.cell(row=i, column=5, value=f"=SOMARPRODUTO(("+idade_rng+f">A{i})*("+classe_rng+"=1))").border = border
    ws4.cell(row=i, column=6, value=f"=B{i}+C{i}").border = border
    ws4.cell(row=i, column=7, value=f"=D{i}+E{i}").border = border
    ws4.cell(row=i, column=8, value=f"=F{i}+G{i}").border = border

for col in range(1, 9):
    ws4.column_dimensions[get_column_letter(col)].width = 24

# Visualizacao (matrizes 1/0 para entender B2 em Contagens)
wsV = wb.create_sheet('Visualizacao')
wsV['A1'] = 'Visualizacao da formula B2 (Left_0): SOMARPRODUTO((Idade<=T)*(Classe=0)) para a linha 2 da aba Contagens'
wsV['A1'].font = Font(b=True)
wsV['A3'] = 'Threshold T (Contagens!A2)'
wsV['B3'] = '=Contagens!A2'
headers_vis = ['#', 'Idade (Dados!B)', 'Classe (Dados!C)', '(Idade<=T)', '(Classe=0)', 'Produto (AND)']
start_row = 5
for j, h in enumerate(headers_vis, start=1):
    c = wsV.cell(row=start_row, column=j, value=h)
    c.font = bold; c.alignment = center; c.fill = header_fill; c.border = border
for idx in range(6):
    r_excel = 2 + idx
    row_excel = start_row + 1 + idx
    wsV.cell(row=row_excel, column=1, value=idx+1).border = border
    wsV.cell(row=row_excel, column=2, value=f'=Dados!B{r_excel}').border = border
    wsV.cell(row=row_excel, column=3, value=f'=Dados!C{r_excel}').border = border
    wsV.cell(row=row_excel, column=4, value=f'=SE(Dados!B{r_excel}<=Contagens!A2,1,0)').border = border
    wsV.cell(row=row_excel, column=5, value=f'=SE(Dados!C{r_excel}=0,1,0)').border = border
    wsV.cell(row=row_excel, column=6, value=f'=D{row_excel}*E{row_excel}').border = border
sum_row = start_row + 1 + 6
wsV.cell(row=sum_row, column=5, value='Soma Produtos =').font = bold
wsV.cell(row=sum_row, column=6, value=f'=SOMA(F{start_row+1}:F{start_row+6})').border = border
wsV.cell(row=sum_row+1, column=5, value='Deve ser igual a Contagens!B2:').font = bold
wsV.cell(row=sum_row+1, column=6, value='=Contagens!B2').border = border
for col in range(1, 7):
    wsV.column_dimensions[get_column_letter(col)].width = 22

# Gini (pt-BR, vírgulas)
ws5 = wb.create_sheet('Gini')
headers_g = ['Threshold T', 'n_left', 'n_right', 'n_total',
             'Left_0', 'Left_1', 'Right_0', 'Right_1',
             'p0_left', 'p1_left', 'Gini_left',
             'p0_right', 'p1_right', 'Gini_right', 'Gini_ponderado']
ws5.append(headers_g)
for idx in range(1, len(headers_g)+1):
    cell = ws5.cell(row=1, column=idx)
    cell.font = bold; cell.alignment = center; cell.fill = header_fill; cell.border = border
for i in range(2, 2+len(midpoints)):
    ws5.cell(row=i, column=1, value=f"=Contagens!A{i}").border = border
    ws5.cell(row=i, column=2, value=f"=Contagens!F{i}").border = border
    ws5.cell(row=i, column=3, value=f"=Contagens!G{i}").border = border
    ws5.cell(row=i, column=4, value=f"=Contagens!H{i}").border = border
    ws5.cell(row=i, column=5, value=f"=Contagens!B{i}").border = border
    ws5.cell(row=i, column=6, value=f"=Contagens!C{i}").border = border
    ws5.cell(row=i, column=7, value=f"=Contagens!D{i}").border = border
    ws5.cell(row=i, column=8, value=f"=Contagens!E{i}").border = border
    ws5.cell(row=i, column=9,  value=f"=SE(B{i}=0,0,E{i}/B{i})").border = border
    ws5.cell(row=i, column=10, value=f"=SE(B{i}=0,0,F{i}/B{i})").border = border
    ws5.cell(row=i, column=11, value=f"=1-((I{i})^2+(J{i})^2)").border = border
    ws5.cell(row=i, column=12, value=f"=SE(C{i}=0,0,G{i}/C{i})").border = border
    ws5.cell(row=i, column=13, value=f"=SE(C{i}=0,0,H{i}/C{i})").border = border
    ws5.cell(row=i, column=14, value=f"=1-((L{i})^2+(M{i})^2)").border = border
    ws5.cell(row=i, column=15, value=f"=(B{i}/D{i})*K{i} + (C{i}/D{i})*N{i}").border = border
for col in range(1, 16):
    ws5.column_dimensions[get_column_letter(col)].width = 18

# Resumo (vírgulas)
ws6 = wb.create_sheet('Resumo')
ws6['A1'] = 'Resumo do Melhor Corte (nó atual)'
ws6['A1'].font = Font(b=True, size=13)
ws6['A3'] = 'Menor Gini ponderado:'
ws6['B3'] = '=MÍNIMO(Gini!O2:O6)'
ws6['A4'] = 'Linha (posição) no quadro de Gini:'
ws6['B4'] = '=CORRESP(B3,Gini!O2:O6,0)'
ws6['A5'] = 'Threshold ótimo (T*):'
ws6['B5'] = '=ÍNDICE(Gini!A2:A6,B4)'
ws6['A7'] = 'Classe majoritária à esquerda (<= T*):'
ws6['B7'] = '=SE(ÍNDICE(Gini!F2:F6,B4)>=ÍNDICE(Gini!E2:E6,B4),1,0)'
ws6['A8'] = 'Classe majoritária à direita (> T*):'
ws6['B8'] = '=SE(ÍNDICE(Gini!H2:H6,B4)>=ÍNDICE(Gini!G2:G6,B4),1,0)'
ws6['A10'] = 'Regra aprendida (neste nó):'
ws6['A11'] = 'SE Idade <= T* ENTÃO prever classe:'
ws6['B11'] = '=B7'
ws6['A12'] = 'SENÃO (Idade > T*) prever classe:'
ws6['B12'] = '=B8'
for r in range(3, 13):
    ws6[f'A{r}'].font = bold
    ws6[f'A{r}'].border = border
    ws6[f'B{r}'].border = border
ws6.column_dimensions['A'].width = 40
ws6.column_dimensions['B'].width = 28
ws6['B3'].fill = highlight_fill
ws6['B5'].fill = highlight_fill
ws6['B11'].fill = highlight_fill

# Predições de exemplo e input do usuário
wsP = wb.create_sheet('Predicoes')
wsP['A1'] = 'Simulador de previsões (usando T* da aba Resumo)'
wsP['A1'].font = Font(b=True, size=13)

# Bloco 1: exemplos fixos
wsP['A3'] = 'Exemplos'
wsP['A4'] = 'Idade'
wsP['B4'] = 'Classe prevista'
exemplos = [27, 31.5, 40]
for idx, idade_val in enumerate(exemplos, start=5):
    wsP[f'A{idx}'] = idade_val
    wsP[f'B{idx}'] = f"=SE(A{idx}<=Resumo!B5,Resumo!B7,Resumo!B8)"
    wsP[f'A{idx}'].border = border
    wsP[f'B{idx}'].border = border
wsP.column_dimensions['A'].width = 16
wsP.column_dimensions['B'].width = 18

# Bloco 2: entrada do usuário
wsP['D3'] = 'Digite uma idade e veja a classe prevista'
wsP['D4'] = 'Idade (entrada)'
wsP['E4'] = 'Classe prevista'
wsP['D5'] = 30
wsP['E5'] = '=SE(D5<=Resumo!B5,Resumo!B7,Resumo!B8)'
wsP['D5'].border = border
wsP['E5'].border = border

# Observação
wsP['A8'] = 'Regra: SE Idade <= T* (Resumo!B5) ENTÃO classe = Resumo!B7; SENÃO classe = Resumo!B8.'

ws6['B12'].fill = highlight_fill

file_name = 'CART_passo_a_passo_ptBR_Excel2010_virgulas_2.xlsx'
wb.save(file_name)
print(f'Arquivo Excel criado: {file_name}')

Arquivo Excel criado: CART_passo_a_passo_ptBR_Excel2010_virgulas_2.xlsx
